In [1]:
from ore_algebra import *

In [2]:
# Create the differential algebra to encode linear differential equation
Pols.<z> = PolynomialRing(QQ)
Diff.<Dz> = OreAlgebra(Pols)

In [3]:
diffWalk = (4*z^2 - z)*Dz^2 + (14*z - 2)*Dz + 6
print(diffWalk)

(4*z^2 - z)*Dz^2 + (14*z - 2)*Dz + 6


La théorie implique que toute singularité d'une solution de `diffWalk` doit être solution de son coefficient dominant.

In [4]:
roots = diffWalk.leading_coefficient().roots(QQbar, multiplicities=False)
roots.sort(key=abs)
print(roots)

[0, 0.25000000000000000?]


In [5]:
diffWalk.local_basis_monomials(0)

[1/z, 1]

In [6]:
first_root = roots[0]
local_basis = diffWalk.local_basis_expansions(first_root, order = 4)
print(local_basis)

[z^(-1), 1 + 3*z + 10*z^2]


On nomme $A_1 (z) = z^{-1}$ et $A_2 (z) = 1 + 3z + 10z^2 + \dots$.

Comme $W$ est convergente au voisinage de 0, on a en fait $W = A_2$.

In [7]:
B = diffWalk.local_basis_expansions(1/4, order = 4)
print(B)

[(z - 1/4)^(-1/2) - 4*(z - 1/4)^(1/2) + 16*(z - 1/4)^(3/2) - 64*(z - 1/4)^(5/2), 1 - 4*(z - 1/4) + 16*(z - 1/4)^2 - 64*(z - 1/4)^3]


On pose

$B_1 (z) = {\left(z - \frac{1}{4}\right)}^{- 1 / 2} - 4 {\left(z - \frac{1}{4}\right)}^{1 / 2} + \dots$

$B_2 (z) = 1 - 4 (z - \frac{1}{4}) + \dots$.

Il faut trouver la matrice de passage de $(A)$ vers $(B)$.

In [8]:
# Represent W in terms of the A_j basis
ini = [0, 1]

# Find numeric coefficients of W in the B_j basis
trans_mat = diffWalk.numerical_transition_matrix([0, 1/4])
bas = trans_mat * vector(ini)

In [9]:
eps = 1e-30
for root in roots[1:2]:
    trans_matrix = diffWalk.numerical_transition_matrix([0, 1/4], eps=eps)
    coeffs_in_local_basis = trans_mat * vector(ini)
    
    if any(not coeff.is_nonzero() for coeff in coeffs_in_local_basis):
        print('WARNING : ignoring some coefficients that appear to be zero')
    
    contributing = [(coeff, expr) for coeff, expr in zip(coeffs_in_local_basis, B)
                    if coeff.is_nonzero()]
    
    print('contributing:', contributing)
    print((contributing[0][1][0]))
    # TODO ordonner les coeffs par croissance

contributing: [([1.0000000000000000 +/- 6.94e-18]*I, (z - 1/4)^(-1/2) - 4*(z - 1/4)^(1/2) + 16*(z - 1/4)^(3/2) - 64*(z - 1/4)^(5/2)), ([-2.0000000000000000 +/- 1.39e-17], 1 - 4*(z - 1/4) + 16*(z - 1/4)^2 - 64*(z - 1/4)^3)]
(1, (z - 1/4)^(-1/2))
